In [1]:
#default_exp anime.ugatit

In [2]:
#export
from functools import partial
import numpy as np
from pathlib import Path
from fastai2.torch_basics import *
import sys
class fakemodule(object):
    def method(a):
        return a
sys.modules["cv2"] = fakemodule
from anime.ugatit_networks import ResnetGenerator,Discriminator,RhoClipper
#from UGATITimport ResnetGenerator,Discriminator,RhoClipper

In [3]:
path=Path("/home/fast/fastai2/anime/")
x=np.load(str(path/'ugatit.npy'))

In [4]:
#export
class UGATIT(object) :
    def __init__(self):
        self.ch = 64
        self.lr=0.0001
        self.weight_decay=0.0001
        """ Weight """
        self.cycle_weight = 10
        self.identity_weight = 10
        self.cam_weight = 1000

        """ Generator """
        self.n_res = 4

        """ Discriminator """
        self.n_dis = 6

        self.img_size = 128
        self.img_ch = 3

    def build_model(self):

        """ Define Generator, Discriminator """
        genA2B = ResnetGenerator(input_nc=3, output_nc=3, ngf=self.ch, n_blocks=self.n_res, img_size=self.img_size, light=False)
        genB2A = ResnetGenerator(input_nc=3, output_nc=3, ngf=self.ch, n_blocks=self.n_res, img_size=self.img_size, light=False)
        disGA = Discriminator(input_nc=3, ndf=self.ch, n_layers=7)
        disGB = Discriminator(input_nc=3, ndf=self.ch, n_layers=7)
        disLA = Discriminator(input_nc=3, ndf=self.ch, n_layers=5)
        disLB = Discriminator(input_nc=3, ndf=self.ch, n_layers=5)
        genA2B.name='genA2B'
        genB2A.name='genB2A'
        disGA.name='disGA'
        disGB.name='disGB'
        disLA.name='disLA'
        disLB.name='disLB'
        self.models=(genA2B,genB2A,disGA,disGB,disLA,disLB)
        

        """ Define Loss """
        L1_loss = nn.L1Loss()
        MSE_loss = nn.MSELoss()
        BCE_loss = nn.BCEWithLogitsLoss()
        self.losses=(L1_loss,MSE_loss,BCE_loss)

        """ Trainer """
        G_optim = torch.optim.Adam(itertools.chain(genA2B.parameters(), genB2A.parameters()), lr=self.lr, betas=(0.5, 0.999), weight_decay=self.weight_decay)
        D_optim = torch.optim.Adam(itertools.chain(disGA.parameters(), disGB.parameters(), disLA.parameters(), disLB.parameters()), lr=self.lr, betas=(0.5, 0.999), weight_decay=self.weight_decay)
        G_optim.name='gen_optim'
        D_optim.name='disc_optim'
        self.optims=(G_optim,D_optim)
        
        """ Define Rho clipper to constraint the value of rho in AdaILN and ILN"""
        self.Rho_clipper = RhoClipper(0, 1)

In [5]:
#export
class UgatitModel(nn.Module):
    def __init__(self,models, devices=[default_device()]):
        super(UgatitModel, self).__init__()
        keys=['GA2B','GB2A','DA','DB','LA','LB']
        self.models=nn.ModuleDict(zip(keys,models))
        self.optimizing_gen=True
        self.devices=devices
    #xA,xB are refering to the device that the model is on if two gpus
    def forward(self, *x, **kwargs):
        xA,xB=x #hackery depending on pytorch trace flag, input can be different
        x_aA,x_bA=xA[0] if len(xA)==1 else xA
        x_aB,x_bB=xB[0] if len(xB)==1 else xB
        
        if self.training:
            if self.optimizing_gen:
                return self._gen_pass(x_aA,x_bA,x_aB,x_bB)
            else:
                with torch.no_grad():
                    self.models['GA2B'].eval()
                    self.models['GB2A'].eval()
                    ga2b=self.models['GA2B']
                    gb2a=self.models['GB2A']
                    fakeA2B,fakeB2A=ga2b(x_aB)[0],gb2a(x_bA)[0] #storing forward pass not needed?
                    self.models['GA2B'].train()
                    self.models['GB2A'].train()
                return (self.models['DA'](x_aA),self.models['LA'](x_aA),self.models['DB'](x_bB),self.models['LB'](x_bB),
                    self.models['DA'](fakeB2A),self.models['LA'](fakeB2A),self.models['DB'](fakeA2B),self.models['LB'](fakeA2B))
        else: 
            return self._gen_pass(x_aA,x_bA,x_aB,x_bB)
    def _gen_pass(self,x_aA,x_bA,x_aB,x_bB):
        ga2b=self.models['GA2B']
        gb2a=self.models['GB2A']
        fakeA2B,fakeB2A=ga2b(x_aB),gb2a(x_bA)
        return (fakeA2B,fakeB2A,
            gb2a(x_aA),ga2b(x_bB),
            self.models['DA'](fakeB2A[0]),self.models['LA'](fakeB2A[0]),self.models['DB'](fakeA2B[0]),self.models['LB'](fakeA2B[0]),
            gb2a(fakeA2B[0].to(self.devices[0])),ga2b(fakeB2A[0].to(self.devices[-1])))
            #Add support for splitting model across GPUs/hackily

    def to(self,device):
        if(device==torch.device("cpu")):
            super().to(device)
        elif(len(self.devices)<=1):
            super().to(device)
        else:
            self.models['GB2A'].to(self.devices[0])
            self.models['DA'].to(self.devices[0])
            self.models['LA'].to(self.devices[0])
            self.models['GA2B'].to(self.devices[1])
            self.models['DB'].to(self.devices[1])
            self.models['LB'].to(self.devices[1])

In [6]:
#export
class UGATITLoss():
    def __init__(self,cycle_weight = 10,identity_weight = 10,cam_weight = 1000,adv_weight=1):
        store_attr(self, "cycle_weight,identity_weight,cam_weight,adv_weight")
        self.MSE_loss=nn.MSELoss()
        self.L1_loss=nn.L1Loss()
        #self.BCE_loss=nn.BCEWithLogitsLoss()
        self.CCE_loss = torch.nn.CrossEntropyLoss()
        self.loss_funcs=[self.MSE_loss,self.L1_loss,self.CCE_loss]
        self.losses=(self.GeneratorLoss(self),self.DiscriminatorLoss(self))
        
    #xb should be pred, need to figure out what to do here
    def __call__(self,xb,yb):
        return self.generator_loss(xb,yb)
            
    def cam_loss(self,fake_A2B_cam_logit,fake_B2B_cam_logit):
        return self.CCE_loss(fake_A2B_cam_logit, torch.ones(fake_A2B_cam_logit.size()[0],dtype=torch.long,device=fake_A2B_cam_logit.device)) \
            + self.CCE_loss(fake_B2B_cam_logit, torch.zeros(fake_B2B_cam_logit.size()[0],dtype=torch.long,device=fake_B2B_cam_logit.device))

    def ad_loss(self,probs,target_value=1):
        prob, cam_prob, _ = probs
        ad_loss = self.MSE_loss(prob, torch.full_like(prob,fill_value=target_value))
        ad_cam_loss = self.MSE_loss(cam_prob, torch.full_like(cam_prob,fill_value=target_value))
        return  ad_loss + ad_cam_loss
    class GeneratorLoss(nn.Module):
        def __init__(self,ugatit_loss):
            super(UGATITLoss.GeneratorLoss, self).__init__()
            self.ugatit_loss=ugatit_loss
            self.losses=nn.ModuleList(self.ugatit_loss.loss_funcs)
            self.tanh=nn.Tanh()
        def decodes(self,preds):
            return ((TensorImage(preds[0][0]),TensorImage(preds[1][0])),)
        def __call__(self,pred,*yb):
            ybA,ybB=yb
            real_AA,real_BA=ybA
            real_AB,real_BB=ybB
            u=self.ugatit_loss
            fake_A2B, fake_A2B_cam_logit, _ = pred[0]
            fake_B2A, fake_B2A_cam_logit, _ = pred[1]

            fake_A2A, fake_A2A_cam_logit, _ = pred[2]
            fake_B2B, fake_B2B_cam_logit, _ = pred[3]
            
            predDA_on_B2A,predLA_on_B2A,predDB_on_A2B,predLB_on_A2B=pred[4],pred[5],pred[6],pred[7]

            fake_cycle_A,_,_ = pred[8]
            fake_cycle_B,_,_= pred[9]                          
                                           
            loss_A = u.adv_weight * (u.ad_loss(predDA_on_B2A) + u.ad_loss(predLA_on_B2A))
            loss_B = u.adv_weight * (u.ad_loss(predDB_on_A2B) + u.ad_loss(predLB_on_A2B))
            
            loss_A += u.cycle_weight * u.L1_loss(fake_cycle_A,real_AA)
            loss_B += u.cycle_weight * u.L1_loss(fake_cycle_B,real_BB)

            loss_A += u.identity_weight * u.L1_loss(fake_A2A, real_AA)
            loss_B += u.identity_weight * u.L1_loss(fake_B2B, real_BB)

            loss_A += u.cam_weight * self.tanh(u.cam_loss(fake_B2A_cam_logit,fake_A2A_cam_logit))
            loss_B += u.cam_weight * self.tanh(u.cam_loss(fake_A2B_cam_logit,fake_B2B_cam_logit))

            return loss_A+loss_B.to(loss_A.device)

    class DiscriminatorLoss(nn.Module):
        def __init__(self,ugatit):
            super(UGATITLoss.DiscriminatorLoss, self).__init__()
            self.ugatit=ugatit
            self.mse=self.ugatit.MSE_loss
            
        def __call__(self,pred,ybA,ybB):
            u=self.ugatit

            #Need to replace with adversarial loss, three variable ones/zeros_like, img, discriminator
            
            #need to add based on device location.
            loss_A = u.ad_loss(pred[0])
            loss_A += u.ad_loss(pred[1])
            loss_B = u.ad_loss(pred[2])
            loss_B += u.ad_loss(pred[3])
            loss_A += u.ad_loss(pred[4],target_value=0)
            loss_A += u.ad_loss(pred[5],target_value=0)
            loss_B += u.ad_loss(pred[6],target_value=0)
            loss_B += u.ad_loss(pred[7],target_value=0)

            return u.adv_weight * (loss_A+loss_B.to(loss_A.device))

In [7]:
ugatit=UGATIT()

In [8]:
ugatit.build_model()

In [9]:
shard_model=UgatitModel(ugatit.models,devices=(torch.device('cuda'),torch.device('cuda',index=1)))
shard_model.to([0,0])

In [10]:
fx=torch.FloatTensor(x)[0][0][None].cuda()
fx.shape

torch.Size([1, 3, 128, 128])

In [11]:
fx=torch.FloatTensor(x)[0][0][None].cuda()
input=((fx,fx),(fx.to(torch.device('cuda',index=1)),fx.to(torch.device('cuda',index=1))))
#pred=shard_model(*input)
#loss_func=UGATITLoss()
#losses=loss_func.losses[0](pred,*input)
#losses.backward()

In [12]:
#for model in ugatit.models:
#    model.load_state_dict(torch.load(path/(model.name+'.pt')))

In [13]:
loss=UGATITLoss()
discriminator_loss=loss.losses[1]
generator_loss=loss.losses[0]
fx=torch.FloatTensor(x)

In [14]:
[ model.name for model in ugatit.models+ugatit.optims ]

['genA2B',
 'genB2A',
 'disGA',
 'disGB',
 'disLA',
 'disLB',
 'gen_optim',
 'disc_optim']

## Need to fix BELOW!!!

for model in ugatit.models: model.train()
ugatit.optims[1].zero_grad()
l=discriminator_loss((fx[0],fx[1]))
print(l.item())
l.backward()
ugatit.optims[1].step()
ugatit.optims[0].zero_grad()
l=generator_loss(fx[0],fx[1])
print(l.item())
l.backward()
ugatit.optims[0].step()
for model in ugatit.models:
    if(model.name=='genA2B' or model.name=='genB2A'):
        model.apply(ugatit.Rho_clipper)

discriminator_loss((fx[0],fx[1])),generator_loss(fx[0],fx[1])

In [15]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted Adam Uses too much gpu memory.ipynb.
Converted Dataloading Original.ipynb.
Converted Dataloading-Copy1.ipynb.
Converted Dataloading-Dec31.ipynb.
Converted Dataloading-Oct22.ipynb.
Converted Dataloading.ipynb.
Converted Dataloading_10_14_2019.ipynb.
Converted Kernel Inception Distance.ipynb.
Converted TensorboardCallback.ipynb.
Converted UGATIT-Dec31Backup.ipynb.
Converted UGATIT-networks.ipynb.
Converted UGATIT.ipynb.
Converted UGATIT_Original.ipynb.
Converted WandbBug.ipynb.
